In [18]:
from nautilus_trader.model.enums import BookAction
from nautilus_trader.model.enums import BookType
from nautilus_trader.model.orderbook.data import OrderBookDeltas
from nautilus_trader.model.orderbook.data import Order
from nautilus_trader.model.orderbook.data import OrderBookDelta
from nautilus_trader.model.orderbook.book import OrderBook
# from nautilus_trader.model.orderbook.book import L2OrderBook
from nautilus_trader.model.identifiers import InstrumentId, Symbol, Venue
from nautilus_trader.backtest.data.providers import TestInstrumentProvider
from collections import defaultdict
from nautilus_trader.examples.strategies.market_maker import MarketMaker
from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.backtest.engine import BacktestEngineConfig
from nautilus_trader.model.identifiers import Venue
from nautilus_trader.model.objects import Money
from nautilus_trader.model.enums import OMSType
from nautilus_trader.model.enums import AccountType
from nautilus_trader.model.currencies import USD, USDT, BTC, APE
from decimal import Decimal

import pandas as pd

ImportError: cannot import name 'APE' from 'nautilus_trader.model.currencies' (/Users/andrewgoldberg/opt/anaconda3/envs/web3/lib/python3.9/site-packages/nautilus_trader/model/currencies.cpython-39-darwin.so)

In [28]:
# bin_btcusdt_ticks_file = ''
# # data_file = '../../data/l2_data_extract_discord/BINANCE-BTC-USDT-incremental_l2_book-2022-10-13-00-00-00-2022-10-14-00-00-00.csv.gz'

data_file = '/Users/andrewgoldberg/Projects/hummingbot-backtest/data/l2_data_extract_discord/BINANCE-BNB-USDT-incremental_l2_book-2022-10-13-00-00-00-2022-10-14-00-00-00.csv.gz'
# data_df = pd.read_csv(data_file, chunksize=100000, compression='gzip', error_bad_lines=False)
data_df = pd.read_csv(data_file, compression='gzip', error_bad_lines=False)

/var/folders/c7/8ppmyxcn6znc4ql4nfmfksy80000gn/T/ipykernel_40322/1328824806.py:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data_df = pd.read_csv(data_file, compression='gzip', error_bad_lines=False)


In [29]:
data_df

,Unnamed: 0,exchange,symbol,timestamp,local_timestamp,is_snapshot,side,price,amount
0,0,BINANCE,BNB-USDT,1665619200029000,1665619200140000,False,bid,270.8,299.883
1,1,BINANCE,BNB-USDT,1665619200029000,1665619200140000,False,bid,270.7,383.521
2,2,BINANCE,BNB-USDT,1665619200029000,1665619200140000,False,bid,270.6,190.845
3,3,BINANCE,BNB-USDT,1665619200029000,1665619200140000,False,bid,270.5,144.141
4,4,BINANCE,BNB-USDT,1665619200029000,1665619200140000,False,bid,270.4,205.904
...,...,...,...,...,...,...,...,...,...
3155346,33814,BINANCE,BNB-USDT,1665705599880000,1665705599990000,False,bid,271.6,79.080
3155347,33815,BINANCE,BNB-USDT,1665705599880000,1665705599990000,False,bid,271.4,172.975
3155348,33816,BINANCE,BNB-USDT,1665705599880000,1665705599990000,False,ask,271.7,465.273
3155349,33817,BINANCE,BNB-USDT,1665705599880000,1665705599990000,False,ask,271.8,159.607


In [24]:
from nautilus_trader.model.currencies import USD, USDT, BTC, BNB
from nautilus_trader.model.instruments.currency_pair import CurrencyPair
from nautilus_trader.model.objects import Money
from nautilus_trader.model.objects import Price
from nautilus_trader.model.objects import Quantity

In [30]:
instrument = CurrencyPair(
            instrument_id=InstrumentId(
                symbol=Symbol("BNBUSDT"),
                venue=Venue("BINANCE"),
            ),
            native_symbol=Symbol("BNBUSDT"),
            base_currency=BTC,
            quote_currency=BNB,
            price_precision=2,
            size_precision=6,
            price_increment=Price(1e-02, precision=2),
            size_increment=Quantity(1e-06, precision=6),
            lot_size=None,
            max_quantity=Quantity(9000, precision=6),
            min_quantity=Quantity(1e-06, precision=6),
            max_notional=None,
            min_notional=Money(10.00000000, USDT),
            max_price=Price(1000000, precision=2),
            min_price=Price(0.01, precision=2),
            margin_init=Decimal(0),
            margin_maint=Decimal(0),
            maker_fee=Decimal("0.001"),
            taker_fee=Decimal("0.001"),
            ts_event=0,
            ts_init=0,
        )

In [31]:
import os, shutil
CATALOG_PATH = os.getcwd() + "/catalog"

# Clear if it already exists, then create fresh
# if os.path.exists(CATALOG_PATH):
#     shutil.rmtree(CATALOG_PATH)
# os.mkdir(CATALOG_PATH)

from nautilus_trader.persistence.catalog import ParquetDataCatalog
catalog = ParquetDataCatalog(CATALOG_PATH)

from nautilus_trader.persistence.external.core import process_files, write_objects
from nautilus_trader.backtest.data.providers import TestInstrumentProvider

# BIN_BTCUSDT = TestInstrumentProvider.btcusdt_binance()
BIN_BNBUSDT = instrument

from nautilus_trader.persistence.external.core import write_objects

write_objects(catalog, [BIN_BNBUSDT])

catalog.instruments()


,id,native_symbol,base_currency,quote_currency,price_precision,size_precision,price_increment,size_increment,lot_size,max_quantity,...,min_notional,max_price,min_price,margin_init,margin_maint,maker_fee,taker_fee,info,ts_event,ts_init
0,BTCUSDT.BINANCE,BTCUSDT,BTC,USDT,2,6,0.01,0.000001,NaN,9000.000000,...,10.00000000 USDT,1000000.00,0.01,0,0,0.001,0.001,None,0,0
2,BNBUSDT.BINANCE,BNBUSDT,BTC,BNB,2,6,0.01,0.000001,NaN,9000.000000,...,10.00000000 USDT,1000000.00,0.01,0,0,0.001,0.001,None,0,0


In [34]:

side_to_code = {
    'bid': 1,
    'ask': 2
}

def parser(data, instrument_id):
    """ Parser function for hist_data FX data, for use with CSV Reader """
    # dt = pd.Timestamp(datetime.datetime.strptime(data['timestamp'].decode(), "%Y%m%d %H%M%S%f"), tz='UTC')
    yield OrderBookDelta(
        instrument_id=instrument_id,
        book_type=BookType.L2_MBP,
        action=BookAction.DELETE if data['amount'].decode()== 0 else BookAction.UPDATE, 
        order=Order(
                    price=int(float(data['price'].decode())),
                    size=int(float(data['amount'].decode())),
                    side=side_to_code[data['side'].decode()],
                    # side=side_to_code[data['side'].decode()]
                ),
                ts_event=int(data['timestamp'].decode()) *1000,
                ts_init=int(data['timestamp'].decode()) *1000

        # ts_event=dt_to_unix_nanos(dt),
        # ts_init=dt_to_unix_nanos(dt),
    )

In [35]:
from nautilus_trader.persistence.external.core import process_files
from nautilus_trader.persistence.external.readers import CSVReader


process_files(
    glob_path=data_file,
    reader=CSVReader(
        block_parser=lambda x: parser(x, instrument_id=BIN_BNBUSDT.id), 
        header=None,
        chunked=False, 
        as_dataframe=False,
    ),
    catalog=catalog,
)


100%|██████████| 1/1 [00:00<00:00, 1077.40it/s]


{'/Users/andrewgoldberg/Projects/hummingbot-backtest/data/l2_data_extract_discord/BINANCE-BNB-USDT-incremental_l2_book-2022-10-13-00-00-00-2022-10-14-00-00-00.csv.gz': 3155351}

In [37]:
# def parser(data_df, instrument_id):
#     alldeltas = []
#     for ts in data_df.timestamp.unique():
#         deltas = []
#         for idx, row in data_df[data_df.timestamp==ts].iterrows():
#             delta_ = OrderBookDelta(
#                 instrument_id=BIN_BTCUSDT.id,
#                 book_type=BookType.L2_MBP,
#                 action=BookAction.DELETE if row.amount == 0 else BookAction.UPDATE,
#                 order=Order(
#                     price=row['price'],
#                     size=row['amount'],
#                     side=side_to_code[row['side']]
#                 ),
#                 ts_event=ts*1000,
#                 ts_init=ts*1000
#             )
#             deltas.append(delta_)
#         alldeltas.append(
#             OrderBookDeltas(
#                 book_type=BookType.L2_MBP,
#                 instrument_id=BIN_BTCUSDT.id,
#                 deltas=deltas,
#                 ts_event=ts*1000,
#                 ts_init=ts*1000,
#             )
#             )
#     return(alldeltas)


In [68]:
# with data_df as reader:
#     reader
#     for chunk in reader:
#         _chunk = parser(chunk)
        

In [69]:
# from nautilus_trader.persistence.catalog import ParquetDataCatalog

# catalog = ParquetDataCatalog("./")
# catalog.instruments()

# write_objects(catalog, alldeltas)
# write_objects(catalog, [BIN_BTCUSDT])

In [70]:
# catalog.order_book_deltas()

In [38]:
catalog.list_data_types()

['currency_pair', 'order_book_data']

In [6]:
BIN_BTCUSDT

CurrencyPair(id=BTCUSDT.BINANCE, native_symbol=BTCUSDT, asset_class=CRYPTO, asset_type=SPOT, quote_currency=USDT, is_inverse=False, price_precision=2, price_increment=0.01, size_precision=6, size_increment=0.000001, multiplier=1, lot_size=None, margin_init=0, margin_maint=0, maker_fee=0.001, taker_fee=0.001, info=None)

In [8]:
from nautilus_trader.backtest.node import BacktestNode, BacktestVenueConfig, BacktestDataConfig, BacktestRunConfig, BacktestEngineConfig


In [41]:
CATALOG_PATH = '/Users/andrewgoldberg/opt/anaconda3/envs/web3/lib/python3.9/site-packages/nautilus_trader/examples/catalog'


In [44]:
from nautilus_trader.model.orderbook.data import OrderBookData

In [45]:
print('start')
# Configure backtest engine
config = BacktestEngineConfig(
    trader_id="BACKTESTER-001",
)
print('config')
# Build the backtest engine
engine = BacktestEngine(config=config)
print('set config')
# Add a trading venue (multiple venues possible)
engine.add_venue(
    venue=BIN_BNBUSDT.venue,
    oms_type=OMSType.NETTING,
    account_type=AccountType.CASH,
    book_type=2,
    # base_currency=BNB,  # Standard single-currency account
    starting_balances=[Money(10_000_000, BNB), Money(10_000_000, USDT)],  # Single-currency or multi-currency accounts
)
print('add venue')
# Add instruments
engine.add_instrument(BIN_BNBUSDT)
print('add instrument')
# Add data
data_config=[
    BacktestDataConfig(
        catalog_path=CATALOG_PATH,
        data_cls=str(OrderBookData.fully_qualified_name()),
        instrument_id=BIN_BNBUSDT.id
    )
]
engine.add_data(catalog)
print('add deltas data')
strategy = MarketMaker(
    instrument_id = BIN_BNBUSDT.id,
    trade_size = Decimal(0.003),
    max_size = Decimal(400_000_000),
)
print('set marketmaker strateg')
# strategy = VolatilityMarketMaker(config=config)
engine.add_strategy(strategy=strategy)
print('add strategy')
# input("Press Enter to continue...")  # noqa (always Python 3)

# Run the engine (from start to end of data)
engine.run_streaming()
print('run streaming')
# Optionally view reports
with pd.option_context(
    "display.max_rows",
    100,
    "display.max_columns",
    None,
    "display.width",
    300,
):
    print(engine.trader.generate_account_report(BIN_BTCUSDT))
    print(engine.trader.generate_order_fills_report())
    print(engine.trader.generate_positions_report())

# For repeated backtest runs make sure to reset the engine
engine.reset()

# Good practice to dispose of the object when done
engine.dispose()

In [ ]:
print('start')
# Configure backtest engine
config = BacktestEngineConfig(
    trader_id="BACKTESTER-001",
)
print('config')
# Build the backtest engine
engine = BacktestEngine(config=config)
print('set config')
# Add a trading venue (multiple venues possible)
engine.add_venue(
    venue=BIN_BTCUSDT.venue,
    oms_type=OMSType.NETTING,
    account_type=AccountType.CASH,
    book_type=2,
    base_currency=BTC,  # Standard single-currency account
    starting_balances=[Money(10_000_000, BTC)],  # Single-currency or multi-currency accounts
)
print('add venue')
# Add instruments
engine.add_instrument(BIN_BTCUSDT)
print('add instrument')
# Add data
data_config=[
    BacktestDataConfig(
        catalog_path=CATALOG_PATH,
        data_class=OrderBookDeltas,
        instrument_id=BIN_BTCUSDT.id
    )
]
engine.add_data(catalog)
print('add deltas data')
strategy = MarketMaker(
    instrument_id = BIN_BTCUSDT.id,
    trade_size = Decimal(0.003),
    max_size = Decimal(400_000_000),
)
print('set marketmaker strateg')
# strategy = VolatilityMarketMaker(config=config)
engine.add_strategy(strategy=strategy)
print('add strategy')
# input("Press Enter to continue...")  # noqa (always Python 3)

# Run the engine (from start to end of data)
engine.run_streaming()
print('run streaming')
# Optionally view reports
with pd.option_context(
    "display.max_rows",
    100,
    "display.max_columns",
    None,
    "display.width",
    300,
):
    print(engine.trader.generate_account_report(BIN_BTCUSDT))
    print(engine.trader.generate_order_fills_report())
    print(engine.trader.generate_positions_report())

# For repeated backtest runs make sure to reset the engine
engine.reset()

# Good practice to dispose of the object when done
engine.dispose()

In [11]:
print('start')
# Configure backtest engine
config = BacktestEngineConfig(
    trader_id="BACKTESTER-001",
)
print('config')
# Build the backtest engine
engine = BacktestEngine(config=config)
print('set config')
# Add a trading venue (multiple venues possible)
venues_config=[
    BacktestVenuyeConfig(
        venue=BIN_BTCUSDT.venue,
        oms_type=OMSType.NETTING,
        account_type=AccountType.CASH,
        book_type=2,
        base_currency=BTC,  # Standard single-currency account
        starting_balances=[Money(10_000_000, BTC)],  # Single-currency or multi-currency accounts
    )
]
data_config=[
    BacktestDataConfig(
        catalog_path=
    )
]
print('add venue')
# Add instruments
engine.add_instrument(BIN_BTCUSDT)
print('add instrument')
# Add data
engine.add_data(alldeltas)
print('add deltas data')
strategy = MarketMaker(
    instrument_id = BIN_BTCUSDT.id,
    trade_size = Decimal(0.003),
    max_size = Decimal(400_000_000),
)
print('set marketmaker strateg')
# strategy = VolatilityMarketMaker(config=config)
engine.add_strategy(strategy=strategy)
print('add strategy')
# input("Press Enter to continue...")  # noqa (always Python 3)

# Run the engine (from start to end of data)
engine.run_streaming()
print('run streaming')
# Optionally view reports
with pd.option_context(
    "display.max_rows",
    100,
    "display.max_columns",
    None,
    "display.width",
    300,
):
    print(engine.trader.generate_account_report(BIN_BTCUSDT))
    print(engine.trader.generate_order_fills_report())
    print(engine.trader.generate_positions_report())

# For repeated backtest runs make sure to reset the engine
engine.reset()

# Good practice to dispose of the object when done
engine.dispose()

SyntaxError: invalid syntax (27470147.py, line 20)